In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
from rich import print
import xarray as xr

In [2]:
pth = pathlib.Path.home() / pathlib.Path("als/cvsbdata", follow_links=True)

In [3]:
pth.resolve()

PosixPath('/Users/kdavis10/als/cvsbdata')

In [4]:
try:
    del cl
except NameError:
    pass

cl = pd.read_csv("/Users/kdavis10/iCloud_Documents/Documents/Personal/Records/csvb/data/Combined Ledger.csv",
                    header=3, parse_dates=["Date"])

In [5]:
cl["Amount"] = cl["Amount"].astype(float)

In [6]:
xcl = cl.set_index("LedgerID").to_xarray()
# Add a dummmy dimension, will make transposing easier later.
xcl = xcl.expand_dims("dummy")

In [7]:
xcl.Amount.data.shape

(1, 1540)

In [8]:
# Reshape so that transpose works.
#amountdata = np.expand_dims(xcl.Amount.data, 0)

# Transfers from one account to another would have opposite signs on the amount.
amount_matches = xcl.Amount.data == -1 * xcl.Amount.data.T

# Dates should be within 5 days.
date_matches = np.abs(xcl.Date.data - xcl.Date.data.T).astype('timedelta64[D]')  < np.timedelta64(5, "D")

# Apply both for list.
matchix = np.where(np.logical_and(amount_matches, date_matches))

# Zip to tuples.
pairs = zip(matchix[0], matchix[1])

# Since (i, j) = (j, i) sort all pairs so that they can be compared, drop items that matched themselves, and use a set to drop duplicates.
pairs = list(set([tuple(np.sort(pair)) for pair in pairs if pair[0] != pair[1]]))

# Get LedgerIDs
#xfer_LIDs = (xcl.isel(LedgerID=xfer_matches[0])["LedgerID"], 
#                 xcl.isel(LedgerID=xfer_matches[1])["LedgerID"])


matcharray = np.array(pairs)

In [9]:
xcl = xcl.squeeze()
left = xcl.isel(LedgerID=matcharray[:,0])
right = xcl.isel(LedgerID=matcharray[:,1])
#left = left.where(left.Amount!=0)
#right = xcl.isel(LedgerID=pairarray[:,1]).sel(Amount=0, drop=True)
#matches = (left, right)

In [10]:
lpandas = left.to_dataframe().reset_index()
rpandas = right.to_dataframe().reset_index()

In [11]:
possible_match_table = pd.merge(lpandas, rpandas, suffixes=["_left", "_right"], left_index=True, right_index=True)

In [12]:
possible_match_table

,LedgerID_left,Ledger_left,Date_left,Description_left,Amount_left,Account_left,LedgerID_right,Ledger_right,Date_right,Description_right,Amount_right,Account_right
0,E5013702,NDFCU:Credit Card,2022-12-02,ASCENDANTCARMICHAEL . SOUTH BEND . IN 0000,-35.00,Medical,43E3E5B7,NDFCU:Credit Card,2022-12-04,FEATHERGILL AND ASSOCIA . MISHAWAKA . IN 0000,35.00,Medical
1,372AD13E,NDFCU:Credit Card,2022-04-27,FEATHERGILL AND ASSOCIA . 574-2821090 . IN 0000,25.00,Medical,13C921A2,NDFCU:Credit Card,2022-04-27,FEATHERGILL AND ASSOCIA . 574-2821090 . IN 0000,-25.00,Medical
2,D4450D58,1st Source:Checking,2023-05-02,Notre Dame FCU - TRANSFER,-3875.48,NDFCU:Credit Card,0ABEADD5,NDFCU:Credit Card,2023-05-02,TRANSFER,3875.48,1st Source:Checking
3,1F29C60A,1st Source:Checking,2022-01-14,Notre Dame FCU - TRANSFER,-1929.62,NDFCU:Credit Card,5513A43F,NDFCU:Credit Card,2022-01-14,TRANSFER,1929.62,1st Source:Checking
4,592523C9,1st Source:Checking,2022-04-08,Notre Dame FCU - TRANSFER,-2146.80,NDFCU:Credit Card,740EC81F,NDFCU:Credit Card,2022-04-08,TRANSFER,2146.80,1st Source:Checking
...,...,...,...,...,...,...,...,...,...,...,...,...
61,4C46682E,1st Source:Checking,2022-05-31,APPLECARD GSBANK PAYMENT,-414.12,Apple:Installments,BC191DC6,Apple:Credit Card,2022-05-30,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,414.12,Electronics
62,B0636345,1st Source:Checking,2023-03-08,ALLY BANK $TRANSFER,-600.00,Ally:Savings,3999FBA2,Ally:Savings,2023-03-10,Requested transfer from KEITH W DAVIS Ally Ban...,600.00,1st Source:Checking
63,1435ECD2,1st Source:Checking,2022-06-28,KEITH W DAVIS P2P,1000.00,Ally:Savings,38931352,Ally:Savings,2022-06-27,Requested transfer to Keith W Davis (1ST SOURC...,-1000.00,1st Source:Checking
64,6798E598,1st Source:Checking,2022-07-06,Notre Dame FCU - TRANSFER,-1986.92,NDFCU:Credit Card,5C3B7381,NDFCU:Credit Card,2022-07-06,TRANSFER,1986.92,1st Source:Checking


In [13]:
lc = [c for c in possible_match_table.columns if c.endswith("_left")]
rc = [c for c in possible_match_table.columns if c.endswith("_right")]
c = list(possible_match_table.columns)
lpd = possible_match_table[lc]
rpd = possible_match_table[rc]

In [14]:
possible_match_table["confirmed"] = np.NaN

In [15]:
possible_match_table

,LedgerID_left,Ledger_left,Date_left,Description_left,Amount_left,Account_left,LedgerID_right,Ledger_right,Date_right,Description_right,Amount_right,Account_right,confirmed
0,E5013702,NDFCU:Credit Card,2022-12-02,ASCENDANTCARMICHAEL . SOUTH BEND . IN 0000,-35.00,Medical,43E3E5B7,NDFCU:Credit Card,2022-12-04,FEATHERGILL AND ASSOCIA . MISHAWAKA . IN 0000,35.00,Medical,NaN
1,372AD13E,NDFCU:Credit Card,2022-04-27,FEATHERGILL AND ASSOCIA . 574-2821090 . IN 0000,25.00,Medical,13C921A2,NDFCU:Credit Card,2022-04-27,FEATHERGILL AND ASSOCIA . 574-2821090 . IN 0000,-25.00,Medical,NaN
2,D4450D58,1st Source:Checking,2023-05-02,Notre Dame FCU - TRANSFER,-3875.48,NDFCU:Credit Card,0ABEADD5,NDFCU:Credit Card,2023-05-02,TRANSFER,3875.48,1st Source:Checking,NaN
3,1F29C60A,1st Source:Checking,2022-01-14,Notre Dame FCU - TRANSFER,-1929.62,NDFCU:Credit Card,5513A43F,NDFCU:Credit Card,2022-01-14,TRANSFER,1929.62,1st Source:Checking,NaN
4,592523C9,1st Source:Checking,2022-04-08,Notre Dame FCU - TRANSFER,-2146.80,NDFCU:Credit Card,740EC81F,NDFCU:Credit Card,2022-04-08,TRANSFER,2146.80,1st Source:Checking,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,4C46682E,1st Source:Checking,2022-05-31,APPLECARD GSBANK PAYMENT,-414.12,Apple:Installments,BC191DC6,Apple:Credit Card,2022-05-30,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,414.12,Electronics,NaN
62,B0636345,1st Source:Checking,2023-03-08,ALLY BANK $TRANSFER,-600.00,Ally:Savings,3999FBA2,Ally:Savings,2023-03-10,Requested transfer from KEITH W DAVIS Ally Ban...,600.00,1st Source:Checking,NaN
63,1435ECD2,1st Source:Checking,2022-06-28,KEITH W DAVIS P2P,1000.00,Ally:Savings,38931352,Ally:Savings,2022-06-27,Requested transfer to Keith W Davis (1ST SOURC...,-1000.00,1st Source:Checking,NaN
64,6798E598,1st Source:Checking,2022-07-06,Notre Dame FCU - TRANSFER,-1986.92,NDFCU:Credit Card,5C3B7381,NDFCU:Credit Card,2022-07-06,TRANSFER,1986.92,1st Source:Checking,NaN


In [16]:
confirmed = []
for i, row in enumerate(possible_match_table[possible_match_table["confirmed"] == np.NaN].iloc):
    if row["confirmed"] is not np.Nan:
        s = [f"{c.split('_')[0]}: {vl}, {vr}" for c, vl, vr in zip(lc, row[lc], row[rc])]
        print(s)
    #y = input()
    #if y.lower() in ["y", "yes"]:
    #   confirmed.append(i)
        

In [17]:
possible_match_table["confirmed"] is np.NaN

False

In [19]:
xcl

<xarray.Dataset>
Dimensions:      (LedgerID: 1540)
Coordinates:
  * LedgerID     (LedgerID) object '7B2F4917' '051E10F3' ... '173FDAAB'
Data variables:
    Ledger       (LedgerID) object '1st Source:Checking' ... 'Fidelity:HSA'
    Date         (LedgerID) datetime64[ns] 2021-12-31 2022-01-03 ... 2023-06-02
    Description  (LedgerID) object 'Initial Balance' ... ' PARTIC CONTR CURRE...
    Amount       (LedgerID) float64 5.068e+03 -674.3 2e+03 ... 6.47 -279.2 279.2
    Account      (LedgerID) object 'Initial Balance' ... 'Paycheck:Pretax'

In [20]:
xcl["Ledger"]

<xarray.DataArray 'Ledger' (LedgerID: 1540)>
array(['1st Source:Checking', '1st Source:Checking',
       '1st Source:Checking', ..., 'Fidelity:HSA', 'Fidelity:HSA',
       'Fidelity:HSA'], dtype=object)
Coordinates:
  * LedgerID  (LedgerID) object '7B2F4917' '051E10F3' ... 'EE2F06CD' '173FDAAB'